# 3D model generation

## 1.准备工作

首先，你应该通过github获取本项目代码：

In [ ]:
!git clone https://github.com/jzyustc/digital_human_teleportation.git

然后进入并设定当前目录为项目目录：

In [ ]:
import os
os.chdir("/content/digital_human_teleportation/")
results_path = "results/"
!pwd

安装所需要的环境：

In [ ]:
!nvidia-smi

In [ ]:
os.chdir("/usr/local/cuda/samples/1_Utilities/deviceQuery/")
!sudo make
!./deviceQuery

安装 python3.6 与 pytorch0.4.0

In [ ]:
!sudo apt install python3.6
!update-alternatives --install /usr/local/bin/python3 python3 /usr/bin/python3.6 2
!update-alternatives --install /usr/local/bin/python3 python3 /usr/bin/python3.7 1
!python --version
!wget http://bootstrap.pypa.io/get-pip.py
!sudo python get-pip.py
!pip install -r planercnn/requirements.txt
!pip install torch==0.4.0 -f https://download.pytorch.org/whl/cu90/stable

安装 cuda

In [6]:
!wget https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
!dpkg -i cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
!apt-key add /var/cuda-repo-9-0-local/7fa2af80.pub
!apt-get update
!apt-get install cuda=9.0.176-1

安装 gcc

In [ ]:
!sudo apt install gcc-5 g++-5
!sudo ln -s /usr/bin/gcc-5 /usr/local/cuda/bin/gcc
!sudo ln -s /usr/bin/g++-5 /usr/local/cuda/bin/g++

查看 cuda 版本

In [ ]:
!cat /usr/local/cuda/version.txt

编译 Mask-RCNN

In [14]:
os.chdir("/content/digital_human_teleportation/planercnn/nms/src/cuda/")
!nvcc -c -o nms_kernel.cu.o nms_kernel.cu -x cu -Xcompiler -fPIC -arch=sm_62
os.chdir("../../")
!python build.py
os.chdir("../")
 
os.chdir("/content/digital_human_teleportation/planercnn/roialign/roi_align/src/cuda/")
!nvcc -c -o crop_and_resize_kernel.cu.o crop_and_resize_kernel.cu -x cu -Xcompiler -fPIC -arch=sm_62
os.chdir("../../")
!python build.py
os.chdir("/content/digital_human_teleportation/")

Including CUDA code.
/content/digital_human_teleportation/planercnn/nms
generating /tmp/tmp92dyhsre/_nms.c
setting the current directory to '/tmp/tmp92dyhsre'
running build_ext
building '_nms' extension
creating content
creating content/digital_human_teleportation
creating content/digital_human_teleportation/planercnn
creating content/digital_human_teleportation/planercnn/nms
creating content/digital_human_teleportation/planercnn/nms/src
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -DWITH_CUDA -I/usr/local/lib/python3.6/dist-packages/torch/utils/ffi/../../lib/include -I/usr/local/lib/python3.6/dist-packages/torch/utils/ffi/../../lib/include/TH -I/usr/local/lib/python3.6/dist-packages/torch/utils/ffi/../../lib/include/THC -I/usr/local/cuda/include -I/usr/include/python3.6m -c _nms.c -o ./_nms.o
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-

升级回 pytorch0.4.1

In [ ]:
!pip install torch==0.4.1 -f https://download.pytorch.org/whl/cu90/stable

## 2.运行 Demo

将所需要的模型下载到对应目录：

In [ ]:
# model for PlaneRCNN
!mkdir planercnn/checkpoint/
!mkdir planercnn/checkpoint/planercnn_normal_warping_refine
!gdown --id 15VOcXXgHEH1naxGiMd2FFjgKmLOdcMEk -O planercnn/checkpoint/planercnn_normal_warping_refine/checkpoint_refine.pth
!gdown --id 1N1XflJeH0ZmAPbaGStdAgv_mJPsI6aLk -O planercnn/checkpoint/planercnn_normal_warping_refine/checkpoint.pth

运行代码

In [16]:
os.chdir("/content/digital_human_teleportation/planercnn/")
!mkdir results/3d_models/
!python bg2ply.py --methods=f --suffix=warping_refine --dataset=inference --customDataFolder=example_images
os.chdir("/content/digital_human_teleportation/")

/content/digital_human_teleportation/planercnn/models/model.py:1473: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(m.weight)
rm: cannot remove 'results/3d_models//*_final.png': No such file or directory
THCudaCheck FAIL file=/pytorch/aten/src/THC/THCGeneral.cpp line=663 error=11 : invalid argument
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1890: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1961: UserWarning: Default upsampling behavior when mode=bilinear is cha

如果想要对自己的图片运行，应将图像放在 /content/digital_human_teleportation/planercnn/example_images/下，从 /content/digital_human_teleportation/planercnn/results/3d_models/下得到对应 .ply 文件。

也可以修改 bg2ply.py 的参数，自定义输入输出目录。